In [ ]:
### Figure 2D
## Create piecewise linear plots to determine a motif score inflection point for each celltype enhancer combination

library(ggplot2)
library(reshape2)

model_list <- list()
for(celltype in unique(emvar_scores_motif_df$celltype)){
  message(celltype)
  model_list[[celltype]] <- list()
  for(enh in enh_all){
    message(enh)
    model_list[[celltype]][[enh]]
    cell_enh_score <- emvar_scores_motif_df$score[which(emvar_scores_motif_df$celltype==celltype & emvar_scores_motif_df$enhancer==enh)]
    cell_enh_skew <- emvar_scores_motif_df$LogSkew[which(emvar_scores_motif_df$celltype==celltype & emvar_scores_motif_df$enhancer==enh)]
    lin_mod <- lm(cell_enh_skew ~ cell_enh_score)
    seg_mod <- segmented(lin_mod, seg.Z=~cell_enh_score, psi = c(20))
    model_list[[celltype]][[enh]]$linear <- lin_mod
    model_list[[celltype]][[enh]]$segmented <- seg_mod
  }
}

reg_lines_plot_both <- function(celltype, enh, df){
  group_temp <- df[which(df[,"celltype"]==celltype & df[,"enhancer"]==enh),c("celltype","enhancer","silencer","LogSkew","score","skew_group")]
  
  piecewise_fitted <- fitted(model_list[[celltype]][[enh]]$segmented)
  linear_fitted <- fitted(model_list[[celltype]][[enh]]$linear)
  
  message("Fitted Models Complete")
  
  p.model.temp <- data.frame(score=group_temp$score, LogSkew = piecewise_fitted)
  l.model.temp <- data.frame(score=group_temp$score, LogSkew = linear_fitted)
  
  message(colnames(p.model.temp))
  
  message("DFs set up")
  p <- ggplot(group_temp, aes(x=score, y=LogSkew)) + geom_point(aes(col=skew_group), alpha=0.5, size=0.5) + xlim(0,35) + 
    geom_line(l.model.temp, mapping=aes(x=score, y=LogSkew), col="purple", lwd=0.6) + 
    geom_line(p.model.temp, mapping=aes(x=score, y=LogSkew), col="salmon", lwd=0.6) +
    theme_light(base_size = 6) + theme(legend.position = "none", axis.title.y = element_text(size=4),
                                       axis.title.x = element_text(size = 4), axis.text = element_text(size=3))
  return(p)
}

for(celltype in names(model_list)){
  message(celltype)
  for(enh in names(model_list[[celltype]])){
    message(enh)
    p <- reg_lines_plot_both(celltype, enh, emvar_scores_motif_df)
    
    pdf(paste0("plots/piecewise_v_linear/", celltype, "_", enh, "_segmented.pdf"), width = 45/25.4, height = 45/25.4)
    print(p)
    dev.off()
  }
}

In [ ]:
###Figure 2E
## Get lists of oligos which are above and below the motif inflection score

oligo_split_list <- list()
for(celltype in names(split_lower)){
  oligo_split_list[[celltype]] <- list()
  for(enh in names(split_lower[[celltype]])){
    oligo_split_list[[celltype]][[enh]] <- list()
    split_val <- 20.86
    if(split_val < 25 & split_val > 15){
      oligo_split_list[[celltype]][[enh]]$below <- emvar_scores_motif_df$silencer[which(emvar_scores_motif_df$celltype==celltype &
                                                                                          emvar_scores_motif_df$enhancer==enh &
                                                                                          emvar_scores_motif_df$score < split_val)]
      oligo_split_list[[celltype]][[enh]]$above <- emvar_scores_motif_df$silencer[which(emvar_scores_motif_df$celltype==celltype &
                                                                                          emvar_scores_motif_df$enhancer==enh &
                                                                                          emvar_scores_motif_df$score >= split_val)]
    }
    if(split_val > 25 | split_val < 15){
      oligo_split_list[[celltype]][[enh]]$below <- emvar_scores_motif_df$silencer[which(emvar_scores_motif_df$celltype==celltype &
                                                                                          emvar_scores_motif_df$enhancer==enh )]
    }
  }
}


## Create box plots over swarm plots for each celltype enhancer combination of the log2FoldChange and motif positive/negative - score above and below categories
## Calculate significance for each celltype enhancer combination and category in comparison to the negative controls

scr_skew_stats <- list()
for(enh in enh_all){
  message(enh)
  scale_max <- 9
  if(enh=="En02"){
    scale_min <- -6
  }
  if(enh=="En09"){
    scale_min <- -2
  }
  if(enh=="En11"){
    scale_min <- -5
  }
  if(enh=="En19"){
    scale_min <- -2
    scale_max <- 8
  }
  if(enh=="En21"){
    scale_min <- 0
    scale_max <- 10
  }
  to_plot <- as.data.frame(all_38_enh_res[[enh]])
  to_plot <- to_plot[which(to_plot$proj %in% c("NegCtrl","motif_ChIP_pos","motif_ChIP_neg")),]
  # plot_split <- colsplit(rownames(to_plot),"\\^", c("enhancer","silencer"))
  # sil_split <- colsplit(plot_split$silencer,"\\.", c("silencer","rep"))
  # to_plot$sil <- sil_split$silencer
  to_plot$group <- "NoScore"
  to_plot$proj <- factor(to_plot$proj, levels = c("NegCtrl","motif_ChIP_pos","motif_ChIP_neg"))
  # to_plot$group <- factor(to_plot$group, levels = c("above", "below", "NoScore"))
  for(celltype in unique(to_plot$celltype)){
    message(celltype)
    for(type in names(oligo_split_list[[celltype]][[enh]])){
      message(type)
      to_plot$group[which(to_plot$sil %in% oligo_split_list[[celltype]][[enh]][[type]] & to_plot$celltype==celltype)] <- type
    }
  }
  to_plot$group <- factor(to_plot$group, levels = c("above", "below", "NoScore"))
  to_plot$group[which(to_plot$proj!="NegCtrl" & to_plot$group=="NoScore")] <- "below"
  to_plot$inter <- interaction(to_plot$proj, to_plot$group)
  to_plot$inter <- factor(to_plot$inter, levels = c("NegCtrl.NoScore", 
                                                    "motif_ChIP_pos.below","motif_ChIP_pos.above",
                                                    "motif_ChIP_neg.below","motif_ChIP_neg.above"))
  scr_skew_stats[[enh]] <- list()
  for(celltype in unique(to_plot$celltype)){
    # message(celltype)
    to_stat <- to_plot
    to_stat <- remove.factors(to_stat)
    stat_temp <- wilcox_test(to_stat[which(to_stat$celltype==celltype),], log2FoldChange~inter, ref.group = "NegCtrl.NoScore", p.adjust.method = "BH")
    scr_skew_stats[[enh]][[celltype]] <- as.data.frame(stat_temp)
  }
  
  med_plot <- to_plot[which(to_plot$proj=="NegCtrl"),] %>%
    group_by(celltype) %>%
    summarise(med_line = median(log2FoldChange, na.rm = T))
  
  pdf(paste0("plots/log2foldchange_box_plots/",enh,"_piecewise_box_swarm_alpha.pdf"), width=174/25.4, height = (174/3.25)/25.4)
  print(ggplot(to_plot, aes(x=inter, y=log2FoldChange, fill=inter, group=inter)) + geom_hline(data=med_plot, aes(yintercept=med_line)) + geom_quasirandom(aes(col=inter), outlier.shape=NA, size=0.5, method = "smiley", alpha=0.2) + geom_boxplot(outlier.shape = NA) +
          ylim(scale_min,scale_max) + facet_grid(~celltype) + theme_light(base_size = 6) + xlab("") + ylab("log2FoldChange") +
          theme(axis.text.x = element_blank(), legend.position = "bottom", strip.background = element_blank(), strip.text.x = element_blank(),legend.margin=margin(t = -.5, unit='cm'), panel.grid = element_line(size = 0.24), panel.border = element_rect(size = 0.24), legend.text = element_text(size=4)) +
          guides(fill=guide_legend(nrow = 1), shape=guide_legend(override.aes = list(size = 0.5)), color=guide_legend(override.aes = list(size = 0.5))) +
          scale_fill_manual(values = c("grey","#66C2A5","#FC8D62","#8DA0CB","#E78AC3","#A6D854","#FFD92F")) + scale_color_manual(values = c("grey","#66C2A5","#FC8D62","#8DA0CB","#E78AC3","#A6D854","#FFD92F")))
  dev.off()
}


model_comp_sum$improvement <- model_comp_sum$seg_r.squared/model_comp_sum$lin_r.squared
model_comp_sum$adj.imp <- model_comp_sum$seg_adj.r.squared/model_comp_sum$lin_adj.r.squared

breakpoints_sd <- sd(model_comp_sum$seg_breakpoints)
breakpoints_mean <- mean(model_comp_sum$seg_breakpoints)

model_comp_sum[which(abs(model_comp_sum$seg_breakpoints - breakpoints_mean) < breakpoints_sd*2),]


scr_skew_stats_df <- data.frame()
for(enh in enh_all){
  message(enh)
  for(celltype in names(scr_skew_stats[[enh]])){
    message(celltype)
    temp <- data.frame(scr_skew_stats[[enh]][[celltype]])
    temp$celltype <- celltype
    temp$enhancer <- enh
    scr_skew_stats_df <- rbind(scr_skew_stats_df,temp)
  }
}

write.table(scr_skew_stats_df,"results/skew_significance_cell_enh.txt",quote = F, sep = "\t",row.names=F)

In [ ]:
###Figure 4B
## Link silencers with all TFs bound to that silencer

tf_plots <- data.frame()
for(celltype in names(standard_res)){
  temp <- standard_res[[celltype]][standard_res[[celltype]]$project=="motif",c("ID","log2FoldChange")]
  temp$celltype <- celltype
  tf_plots <- rbind(tf_plots,temp)
}

tf_id_split <- colsplit(tf_plots$ID,"\\^",c("enhancer","silencer"))
tf_plots$enhancer <- tf_id_split$enhancer
tf_plots$silencer <- tf_id_split$silencer

tf_plots$tf_stat <- 0
for(celltype in unique(tf_plots$celltype)){
  message(celltype)
  for(sil in unique(tf_plots$silencer)){
    if(any(chip_tf_celltypes[[celltype]][which(chip_tf_celltypes[[celltype]]$name==sil),5:ncol(chip_tf_celltypes[[celltype]])]==1)){
      tf_plots$tf_stat[which(tf_plots$silencer==sil & tf_plots$celltype==celltype)] <- 1
    }
  }
}

tf_plots$tf_stat <- as.factor(tf_plots$tf_stat)
tf_plots$celltype <- as.factor(tf_plots$celltype)

accession_tf_name_map <- read.delim("ENCODE_ChIP/complete_histone_tf_list.txt", stringsAsFactors = F)
colnames(accession_tf_name_map) <- c("Accession","Target","Celltype")

accession_celltype_list <- list()
accession_tf_name_map$Celltype[which(accession_tf_name_map$Celltype=="SK-N-SH")] <- "SKNSH"
accession_tf_name_map$Celltype[which(accession_tf_name_map$Celltype=="HEPG2")] <- "HepG2"
for(celltype in unique(accession_tf_name_map$Celltype)){
  message(celltype)
  accession_celltype_list[[celltype]] <- list()
  for(tf in unique(accession_tf_name_map$Target[which(accession_tf_name_map$Celltype==celltype)])){
    # if(tf=="REST") next
    accession_celltype_list[[celltype]][[tf]] <- accession_tf_name_map$Accession[which(accession_tf_name_map$Target==tf & accession_tf_name_map$Celltype==celltype)]
  }
}

## Count how many transcirption factors bind with each oligo

bound_factor <- tf_plots[,1:5]
bound_factor$REST <- "unbound"
for(celltype in unique(bound_factor$celltype)){
  message(celltype)
  for(sil in unique(bound_factor$silencer[which(bound_factor$celltype==celltype)])){
    if(isTRUE(chip_tf_celltypes[[celltype]][which(chip_tf_celltypes[[celltype]]$name==sil),accession_celltype_list[[celltype]]$REST[1]]==1) | isTRUE(chip_tf_celltypes[[celltype]][which(chip_tf_celltypes[[celltype]]$name==sil),accession_celltype_list[[celltype]]$REST[2]]==1)){
      bound_factor$REST[which(bound_factor$silencer==sil & bound_factor$celltype==celltype)] <- "bound"
    }
  }
}

## Add allelic skew information

scramble_skew <- list()
for(celltype in names(emVAR_all)){
  message(celltype)
  intersect_list <- bound_factor$ID[bound_factor$ID[bound_factor$celltype==celltype] %in% emVAR_all[[celltype]]$ID]
  scramble_skew[[celltype]] <- emVAR_all[[celltype]][emVAR_all[[celltype]]$ID %in% intersect_list,c("ID","B_log2FC","LogSkew")]
}

bound_factor$scr_l2fc <- NA
bound_factor$LogSkew <- NA
for(celltype in names(scramble_skew)){
  message(celltype)
  # bound_factor[which(bound_factor$celltype==celltype),] <- merge(bound_factor[which(bound_factor$celltype==celltype),], scramble_skew[[celltype]], by="ID", all=T)
  for(id in scramble_skew[[celltype]]$ID){
    bound_factor$scr_l2fc[which(bound_factor$celltype==celltype & bound_factor$ID==id)] <- scramble_skew[[celltype]]$B_log2FC[which(scramble_skew[[celltype]]$ID==id)]
    bound_factor$LogSkew[which(bound_factor$celltype==celltype & bound_factor$ID==id)] <- scramble_skew[[celltype]]$LogSkew[which(scramble_skew[[celltype]]$ID==id)]
  }
}

bound_factor$skew_na <- is.na(bound_factor$LogSkew)

bound_factor$num_bound <- NA
bound_tf_names <- list()
for(celltype in names(chip_tf_celltypes)){
  message(celltype)
  bound_tf_names[[celltype]] <- list()
  for(sil in unique(bound_factor$silencer[which(bound_factor$celltype==celltype)])){
    acc_bound <- colnames(chip_tf_celltypes[[celltype]])[chip_tf_celltypes[[celltype]][which(chip_tf_celltypes[[celltype]]$name==sil),]==1]
    tf_bound <- unique(accession_tf_name_map$Target[which(accession_tf_name_map$Accession %in% acc_bound)])
    bound_tf_names[[celltype]][[sil]] <- tf_bound
    bound_factor$num_bound[which(bound_factor$silencer==sil & bound_factor$celltype==celltype)] <- length(tf_bound)
  }
}

# accession_tf_name_map <- read.delim("ENCODE_ChIP/complete_histone_tf_list.txt", stringsAsFactors = F)
# colnames(accession_tf_name_map) <- c("Accession","Target","Celltype")
# 
# accession_celltype_list <- list()
# accession_tf_name_map$Celltype[which(accession_tf_name_map$Celltype=="SK-N-SH")] <- "SK.N.SH"
# accession_tf_name_map$Celltype[which(accession_tf_name_map$Celltype=="HEPG2")] <- "HepG2"
# for(celltype in unique(accession_tf_name_map$Celltype)){
#   message(celltype)
#   accession_celltype_list[[celltype]] <- list()
#   for(tf in unique(accession_tf_name_map$Target[which(accession_tf_name_map$Celltype==celltype)])){
#     # if(tf=="REST") next
#     accession_celltype_list[[celltype]][[tf]] <- accession_tf_name_map$Accession[which(accession_tf_name_map$Target==tf & accession_tf_name_map$Celltype==celltype)]
#   }
# }

## Reorganize list so that it's organized by celltype and transcription factor
tf_sil_list <- list()
for(celltype in names(accession_celltype_list)){
  message(celltype)
  tf_sil_list[[celltype]] <- list()
  temp <- chip_tf_celltypes[[celltype]]
  for(tf in names(accession_celltype_list[[celltype]])){
    temp_sil_list <- c()
    for(acc in accession_celltype_list[[celltype]][[tf]]){
      temp_sil_list <- c(temp_sil_list, chip_tf_celltypes[[celltype]]$name[which(chip_tf_celltypes[[celltype]][,acc]==1)])
    }
    tf_sil_list[[celltype]][[tf]] <- temp_sil_list
  }
}

## Group oligos for each celltype and enhancer in order to categorize where TFs align on the spectrum of REST/TF binding

med_comp_list <- list()
med_comp_list$A_B <- list()
med_comp_list$A_D <- list()
med_comp_list$C_B <- list()
med_comp_list$C_D <- list()
med_comp_list$other <- list()
for(celltype in names(tf_sil_list)){
  message(celltype)
  med_comp_list$A_B[[celltype]] <- list()
  med_comp_list$A_D[[celltype]] <- list()
  med_comp_list$C_B[[celltype]] <- list()
  med_comp_list$C_D[[celltype]] <- list()
  med_comp_list$other[[celltype]] <- list()
  cell_sub <- bound_factor[which(bound_factor$celltype==celltype),1:8]
  for(enh in unique(cell_sub$enhancer)){
    message(enh)
    med_comp_list$A_B[[celltype]][[enh]] <- list()
    med_comp_list$A_D[[celltype]][[enh]] <- list()
    med_comp_list$C_B[[celltype]][[enh]] <- list()
    med_comp_list$C_D[[celltype]][[enh]] <- list()
    med_comp_list$other[[celltype]][[enh]] <- list()
    med_comp_list$A_B[[celltype]][[enh]]$TFs <- c()
    med_comp_list$A_D[[celltype]][[enh]]$TFs <- c()
    med_comp_list$C_B[[celltype]][[enh]]$TFs <- c()
    med_comp_list$C_D[[celltype]][[enh]]$TFs <- c()
    med_comp_list$other[[celltype]][[enh]]$TFs <- c()
    cell_enh_sub <- cell_sub[which(cell_sub$enhancer==enh),]
    for(tf in names(tf_sil_list[[celltype]])){
      temp_sil_list <- tf_sil_list[[celltype]][[tf]]
      RB_TFB <- median(cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="bound" & cell_enh_sub$silencer %in% temp_sil_list)], na.rm = T)
      RU_TFB <- median(cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="unbound" & cell_enh_sub$silencer %in% temp_sil_list)], na.rm = T)
      RB_TFU <- median(cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="bound" & cell_enh_sub$silencer %notin% temp_sil_list)], na.rm = T)
      RU_TFU <- median(cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="unbound" & cell_enh_sub$silencer %notin% temp_sil_list)], na.rm = T)
      if(any(is.na(c(RB_TFB, RU_TFB)))){
        med_comp_list$other[[celltype]][[enh]]$TFs <- c(med_comp_list$other[[celltype]][[enh]]$TFs,tf)
        next
      }
      if(RB_TFB > RB_TFU & RU_TFB > RU_TFU){
        med_comp_list$A_B[[celltype]][[enh]]$TFs <- c(med_comp_list$A_B[[celltype]][[enh]]$TFs,tf)
        med_comp_list$A_B[[celltype]][[enh]]$med_skew <- median(cell_enh_sub$LogSkew[which(cell_enh_sub$silencer %in% temp_sil_list)], na.rm = T)
      }
      else if(RB_TFB > RB_TFU & RU_TFB < RU_TFU){
        med_comp_list$A_D[[celltype]][[enh]]$TFs <- c(med_comp_list$A_D[[celltype]][[enh]]$TFs,tf)
        med_comp_list$A_D[[celltype]][[enh]]$med_skew <- median(cell_enh_sub$LogSkew[which((cell_enh_sub$REST=="bound" & cell_enh_sub$silencer %in% temp_sil_list) | 
                                                                                             (cell_enh_sub$REST=="unbound" & cell_enh_sub$silencer %notin% temp_sil_list))], na.rm = T)
      }
      else if(RB_TFB < RB_TFU & RU_TFB > RU_TFU){
        med_comp_list$C_B[[celltype]][[enh]]$TFs <- c(med_comp_list$C_B[[celltype]][[enh]]$TFs,tf)
        med_comp_list$C_B[[celltype]][[enh]]$med_skew <- median(cell_enh_sub$LogSkew[which((cell_enh_sub$REST=="bound" & cell_enh_sub$silencer %notin% temp_sil_list) | 
                                                                                             (cell_enh_sub$REST=="unbound" & cell_enh_sub$silencer %in% temp_sil_list))], na.rm = T)
      }
      else if(RB_TFB < RB_TFU & RU_TFB < RU_TFU){
        med_comp_list$C_D[[celltype]][[enh]]$TFs <- c(med_comp_list$C_D[[celltype]][[enh]]$TFs,tf)
        med_comp_list$C_D[[celltype]][[enh]]$med_skew <- median(cell_enh_sub$LogSkew[which(cell_enh_sub$silencer %notin% temp_sil_list)], na.rm = T)
      }
    }
  }
}

med_comp_plot_df <- data.frame()
for(celltype in names(tf_sil_list)){
  message(celltype)
  cell_sub <- bound_factor[which(bound_factor$celltype==celltype),1:8]
  for(enh in unique(cell_sub$enhancer)){
    message(enh)
    cell_enh_sub <- cell_sub[which(cell_sub$enhancer==enh),]
    for(tf in names(tf_sil_list[[celltype]])){
      temp_sil_list <- tf_sil_list[[celltype]][[tf]]
      A <- cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="bound" & cell_enh_sub$silencer %in% temp_sil_list)]
      A_med <- median(A, na.rm = T)
      B <- cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="unbound" & cell_enh_sub$silencer %in% temp_sil_list)]
      B_med <- median(B, na.rm = T)
      C <- cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="bound" & cell_enh_sub$silencer %notin% temp_sil_list)]
      C_med <- median(C, na.rm = T)
      D <- cell_enh_sub$log2FoldChange[which(cell_enh_sub$REST=="unbound" & cell_enh_sub$silencer %notin% temp_sil_list)]
      D_med <- median(D, na.rm = T)
      if(length(A[is.na(A)])==length(A)){
        A <- c()
      }
      if(length(B[is.na(B)])==length(B)){
        B <- c()
      }
      if(length(C[is.na(C)])==length(C)){
        C <- c()
      }
      if(length(D[is.na(D)])==length(D)){
        D <- c()
      }
      if(length(A) > 0 & length(C) > 0){
        A_C_sig <- wilcox.test(A,C)$p.value
      }
      if(length(A) == 0 | length(C) == 0){
        A_C_sig <- NA
      }
      if(length(B) > 0 & length(D) > 0){
        B_D_sig <- wilcox.test(B,D)$p.value
      }
      if(length(B) == 0 | length(D) == 0){
        B_D_sig <- NA
      }
      temp <- data.frame(celltype, enh, tf, A_med, B_med, C_med, D_med, A_C_sig, B_D_sig)
      med_comp_plot_df <- rbind(med_comp_plot_df, temp)
    }
  }
}

med_comp_plot_df$delta_AC <- med_comp_plot_df$A_med - med_comp_plot_df$C_med
med_comp_plot_df$delta_BD <- med_comp_plot_df$B_med - med_comp_plot_df$D_med

med_comp_plot_df$A_C_adj <- p.adjust(med_comp_plot_df$A_C_sig, method = "hochberg")
med_comp_plot_df$B_D_adj <- p.adjust(med_comp_plot_df$B_D_sig, method = "hochberg")

med_comp_plot_df$significance <- NA
med_comp_plot_df$significance[which(med_comp_plot_df$A_C_adj < 0.05 & med_comp_plot_df$B_D_adj < 0.05)] <- "both"
med_comp_plot_df$significance[which(med_comp_plot_df$A_C_adj < 0.05 & med_comp_plot_df$B_D_adj > 0.05)] <- "REST+ only"
med_comp_plot_df$significance[which(med_comp_plot_df$A_C_adj > 0.05 & med_comp_plot_df$B_D_adj < 0.05)] <- "REST- only"
med_comp_plot_df$significance[which(med_comp_plot_df$A_C_adj > 0.05 & med_comp_plot_df$B_D_adj > 0.05)] <- "neither"
med_comp_plot_df$significance <- factor(med_comp_plot_df$significance, levels = c("both","REST+ only","REST- only","neither"))

## save plot as pdf or interactive plotly

for(celltype in unique(med_comp_plot_df$celltype)){
  message(celltype)
  for(enh in enh_all){
    message(enh)
    cell_enh_sub <- med_comp_plot_df[which(med_comp_plot_df$celltype==celltype & med_comp_plot_df$enh==enh),]
    max_ac <- max(cell_enh_sub$delta_AC, na.rm=T)
    min_ac <- min(cell_enh_sub$delta_AC, na.rm=T)
    max_bd <- max(cell_enh_sub$delta_BD, na.rm = T)
    min_bd <- min(cell_enh_sub$delta_BD, na.rm = T)
    x_max <- max(abs(max_ac), abs(min_ac))
    y_max <- max(abs(max_bd), abs(min_bd))
    # pdf(paste0("plots/tf_analysis/scatter_plots/",celltype,"_",enh,"_delta_bound_v_unbound_centered_bhadj.pdf"), height=55/25.4, width = 55/25.4)
    a <- ggplot(cell_enh_sub, aes(x=delta_AC, y=delta_BD, col=significance, key=tf)) + geom_point(aes(alpha=0.3), size=2) + geom_hline(yintercept = 0) + geom_vline(xintercept = 0) +
      xlim(-x_max,x_max) + ylim(-y_max,y_max) + xlab("") + ylab(NULL) +
      theme_light(base_size = 6) + scale_color_manual(values = c("red","dodgerblue","green3","black"),name="Significance") +
      theme(legend.position = "bottom", legend.margin=margin(t = -0.6, unit='cm'),legend.justification = "left", legend.key.width = unit(0.6, "mm")) + 
      guides(fill=guide_legend(nrow = 1),shape=guide_legend(override.aes = list(size = 0.5)),color = guide_legend(override.aes = list(size = 0.5)),
             alpha=F, size=F)
    # print(a)
    # dev.off()
    
    a_ly <- ggplotly(a)
    htmlwidgets::saveWidget(as_widget(a_ly), paste0("plots/tf_analysis/scatter_plots/",celltype,"_",enh,"_delta_bound_v_unbound_centered_bhadj.html"))
    
    # pdf(paste0("plots/tf_analysis/scatter_plots/",celltype,"_",enh,"_delta_bound_v_unbound.pdf"), height=55/25.4, width = 55/25.4)
    # print(ggplot(cell_enh_sub, aes(x=delta_AC, y=delta_BD, col=significance)) + geom_point(aes(alpha=0.3), size=2) + geom_hline(yintercept = 0) + geom_vline(xintercept = 0) +
    #         xlim(-2.5,7.25) + ylim(-2.5,7.25) + xlab("") + ylab("") +
    #         theme_light(base_size = 6) + scale_color_manual(values = c("red","dodgerblue","green3","black"),name="Significance") +
    #         theme(legend.position = "bottom", legend.margin=margin(t = -0.5, unit='cm'),legend.justification = "left", legend.key.width = unit(0.7, "mm")) + 
    #         guides(fill=guide_legend(nrow = 1),shape=guide_legend(override.aes = list(size = 0.5)),color = guide_legend(override.aes = list(size = 0.5)),
    #                alpha=F, size=F)) 
    # dev.off()
    
  }
}

In [ ]:
### Figure 4C
## Calculate the odds ratio of each of the categories of TFs

all_data_single_df <- data.frame()
for(celltype in names(standard_res)){
  message(celltype)
  
  id_split <- colsplit(standard_res[[celltype]]$ID, "\\^",c("enhancer","silencer"))
  
  for(enh in unique(id_split$enhancer)){
    message(enh)
    
    temp <- id_split$silencer[which(id_split$enhancer==enh)]
    temp2 <- standard_res[[celltype]][which(id_split$enhancer==enh), c("project","log2FoldChange","lfcSE")]
    
    temp <- cbind(temp, temp2)
    
    colnames(temp) <- c("silencer","project",paste0(celltype,".",enh,".l2FC"), paste0(celltype,".",enh,".l2FC_SE"))
    
    if(celltype == "GM12878" & enh == "En02"){
      all_data_single_df <- temp
    }
    else{
      all_data_single_df <- merge(all_data_single_df, temp, by=c("silencer","project"), all=T)
    }
  }
}

head(all_data_single_df)

write.table(all_data_single_df,"results/silencer_project_l2FC_l2FC.SE_all_cell_enh.txt", quote=F, sep = "\t", row.names = F)

emvar_all_motif_scores <- data.frame()
for(celltype in names(emVAR_all)){
  message(celltype)
  emvar_temp <- emVAR_all[[celltype]][is.na(emVAR_all[[celltype]]$ref_allele),]
  emvar_temp$celltype <- celltype
  emvar_all_motif_scores <- rbind(emvar_all_motif_scores, emvar_temp)
}

head(emvar_all_motif_scores)
motif_all_IDs <- colsplit(emvar_all_motif_scores$ID, "\\^", c("enhancer","silencer"))
emvar_all_motif_scores$enhancer <- motif_all_IDs$enhancer
emvar_all_motif_scores$silencer <- motif_all_IDs$silencer

emvar_all_motif_scores <- merge(emvar_all_motif_scores, ref_all_scores[which(ref_all_scores$start > 90 & ref_all_scores$stop < 112 & ref_all_scores$X.pattern.name=="MA0138.2"),], by.x="silencer", by.y="sequence.name", drop=F)

head(emvar_all_motif_scores)
emvar_all_motif_scores$skew_group <- NA
emvar_all_motif_scores$skew_group[which(emvar_all_motif_scores$LogSkew < 0)] <- 1
emvar_all_motif_scores$skew_group[which(emvar_all_motif_scores$LogSkew > 0)] <- 0


names(emvar_all_motif_scores)

emvar_scores_motif_df <- emvar_all_motif_scores[,c("skew_group","A_Ctrl_Mean","A_Exp_Mean","A_log2FC","A_log2FC_SE","A_logP","A_logPadj_BH","A_logPadj_BF",
                                                   "B_Ctrl_Mean","B_Exp_Mean","B_log2FC","B_log2FC_SE","B_logP","B_logPadj_BH","B_logPadj_BF","LogSkew",
                                                   "Skew_logP","Skew_logFDR","celltype","enhancer","silencer","score")]

emvar_scores_motif_df <- emvar_scores_motif_df[complete.cases(emvar_scores_motif_df$skew_group),]
emvar_scores_motif_df$skew_group <- as.numeric(emvar_scores_motif_df$skew_group)


oligo_split_list <- list()
for(celltype in names(standard_res)){
  oligo_split_list[[celltype]] <- list()
  for(enh in enh_all){
    oligo_split_list[[celltype]][[enh]] <- list()
    # split_val <- model_comp_sum$seg_breakpoints[which(model_comp_sum$celltype==celltype & model_comp_sum$enh==enh)]
    split_val <- 20.86
    if(split_val < 25 & split_val > 15){
      oligo_split_list[[celltype]][[enh]]$below <- emvar_scores_motif_df$silencer[which(emvar_scores_motif_df$celltype==celltype &
                                                                                          emvar_scores_motif_df$enhancer==enh &
                                                                                          emvar_scores_motif_df$score < split_val)]
      oligo_split_list[[celltype]][[enh]]$above <- emvar_scores_motif_df$silencer[which(emvar_scores_motif_df$celltype==celltype &
                                                                                          emvar_scores_motif_df$enhancer==enh &
                                                                                          emvar_scores_motif_df$score >= split_val)]
    }
    if(split_val > 25 | split_val < 15){
      oligo_split_list[[celltype]][[enh]]$below <- emvar_scores_motif_df$silencer[which(emvar_scores_motif_df$celltype==celltype &
                                                                                        emvar_scores_motif_df$enhancer==enh )]
    }
  }
}

### REDO with color relating to significance and positivity of odds ratio as opposed to the actual odds ratio value
or_tf_df <- data.frame()
for(celltype in unique(med_comp_plot_df$celltype)){
  message(celltype)
  for(enh in enh_all){
    message(enh)
    cell_enh_sub <- med_comp_plot_df[which(med_comp_plot_df$celltype==celltype & med_comp_plot_df$enh==enh),]
    
    cell_enh_sub2 <- bound_factor[which(bound_factor$celltype==celltype & bound_factor$enhancer==enh),1:8]
    
    cell_enh_sub$OR <- NA
    cell_enh_sub$OR_sig <- NA
    for(tf in names(tf_sil_list[[celltype]])){
      temp_sil_list <- tf_sil_list[[celltype]][[tf]]
      A <- cell_enh_sub2$log2FoldChange[which(cell_enh_sub2$REST=="bound" & cell_enh_sub2$silencer %in% temp_sil_list & cell_enh_sub2$silencer %in% oligo_split_list[[celltype]][[enh]]$above)]
      
      B <- cell_enh_sub2$log2FoldChange[which(cell_enh_sub2$REST=="unbound" & cell_enh_sub2$silencer %in% temp_sil_list & cell_enh_sub2$silencer %notin% oligo_split_list[[celltype]][[enh]]$above)]
      
      C <- cell_enh_sub2$log2FoldChange[which(cell_enh_sub2$REST=="bound" & cell_enh_sub2$silencer %notin% temp_sil_list & cell_enh_sub2$silencer %in% oligo_split_list[[celltype]][[enh]]$above)]
      
      D <- cell_enh_sub2$log2FoldChange[which(cell_enh_sub2$REST=="unbound" & cell_enh_sub2$silencer %notin% temp_sil_list & cell_enh_sub2$silencer %notin% oligo_split_list[[celltype]][[enh]]$above)]
      
      OR_above_mat <- matrix(c(length(A),length(B),length(C),length(D)), nrow = 2)
      ft_above <- fisher.test(OR_above_mat)
      OR_above_est <- as.numeric(ft_above$estimate)
      OR_above_pval <- as.numeric(ft_above$p.value)
      cell_enh_sub$OR[which(cell_enh_sub$tf==tf)] <- OR_above_est
      cell_enh_sub$OR_sig[which(cell_enh_sub$tf==tf)] <- OR_above_pval
    }
    cell_enh_sub$log2OR <- log2(cell_enh_sub$OR)
    or_tf_df <- rbind(or_tf_df, cell_enh_sub)
    
    max_ac <- max(cell_enh_sub$delta_AC, na.rm=T)
    min_ac <- min(cell_enh_sub$delta_AC, na.rm=T)
    max_bd <- max(cell_enh_sub$delta_BD, na.rm = T)
    min_bd <- min(cell_enh_sub$delta_BD, na.rm = T)
    x_max <- max(abs(max_ac), abs(min_ac))
    y_max <- max(abs(max_bd), abs(min_bd))
    pdf(paste0("plots/tf_analysis/scatter_plots/odds_ratio/",celltype,"_",enh,"_delta_bound_v_unbound_centered.pdf"), height=55/25.4, width = 55/25.4)
    print(ggplot(cell_enh_sub, aes(x=delta_AC, y=delta_BD, col=log2OR, shape=significance, key=tf)) + geom_point(aes(alpha=0.3), size=1) + geom_hline(yintercept = 0) + geom_vline(xintercept = 0) +
      xlim(-x_max,x_max) + ylim(-y_max,y_max) + xlab("") + ylab("") +
      theme_light(base_size = 6) + scale_shape_manual(values = c("both"=15,"REST+ only"=18,"REST- only"=17, "neither"=16),name="Significance") + scale_color_gradient(low="blue",high="orange") +
      theme(legend.position = "bottom", legend.margin=margin(t = -0.5, unit='cm'),legend.justification = "left", legend.key.width = unit(0.7, "mm"), legend.box = "vertical") +
      guides(shape=guide_legend(override.aes = list(size = 0.5)),color = guide_legend(override.aes = list(size = 0.5)),
             alpha=F, size=F))
    dev.off()
    
    pdf(paste0("plots/tf_analysis/scatter_plots/odds_ratio/",celltype,"_",enh,"_delta_bound_v_unbound.pdf"), height=55/25.4, width = 55/25.4)
    print(ggplot(cell_enh_sub, aes(x=delta_AC, y=delta_BD, col=log2OR, shape=significance, key=tf)) + geom_point(aes(alpha=0.3), size=1) + geom_hline(yintercept = 0) + geom_vline(xintercept = 0) +
            xlim(-2.5,7.25) + ylim(-2.5,7.25) + xlab("") + ylab("") +
            theme_light(base_size = 6) + scale_shape_manual(values = c("both"=15,"REST+ only"=18,"REST- only"=17, "neither"=16),name="Significance") + scale_color_gradient(low="blue",high="orange") +
            theme(legend.position = "bottom", legend.margin=margin(t = -0.5, unit='cm'),legend.justification = "left", legend.key.width = unit(0.7, "mm"), legend.box = "vertical") +
            guides(shape=guide_legend(override.aes = list(size = 0.5)),color = guide_legend(override.aes = list(size = 0.5)),
                   alpha=F, size=F))
    dev.off()
  }
}


or_tf_df$quadrant <- 0
or_tf_df$quadrant[which(or_tf_df$delta_AC > 0 & or_tf_df$delta_BD > 0)] <- 1
or_tf_df$quadrant[which(or_tf_df$delta_AC < 0 & or_tf_df$delta_BD > 0)] <- 2
or_tf_df$quadrant[which(or_tf_df$delta_AC < 0 & or_tf_df$delta_BD < 0)] <- 3
or_tf_df$quadrant[which(or_tf_df$delta_AC > 0 & or_tf_df$delta_BD < 0)] <- 4

or_tf_df$sig_or <- ">=0.05"
or_tf_df$sig_or[which(or_tf_df$OR_sig < 0.05)] <- "<0.05"

## Create interactive plotly version, or pdf of Odds Ratio significance plot

for(celltype in unique(or_tf_df$celltype)){
  message(celltype)
  for(enh in enh_all){
    message(enh)
    to_plot <- or_tf_df[which(or_tf_df$celltype==celltype & or_tf_df$enh==enh & or_tf_df$significance=="both"),]
    to_plot$quadrant <- factor(to_plot$quadrant, levels=c("1","2","3","4"))
    # pdf(paste0("plots/tf_analysis/jitter/",celltype,"_",enh,"_OR_sig_quadrant_both_only.pdf"), height=55/25.4, width = 55/25.4)
    
    tmp <- ggplot(to_plot, aes(x=quadrant, y=log2OR, col=sig_or, key=tf)) + geom_jitter(alpha=0.4, size=0.6) + # geom_point(alpha=0.4, size=0.6) +
            scale_color_manual(values=c(">=0.05"="black","<0.05"="red"), name="OR Significance") + theme_light(base_size = 6) + theme(legend.position = "bottom")
    tmply <- ggplotly(tmp)
    
    htmlwidgets::saveWidget(tmply,paste0("plots/tf_analysis/jitter/plotly/",celltype,"_",enh,"_OR_sig_quadrant_both_only.html"))
    # dev.off()
  }
}

In [ ]:
###Figure 4D-E
## Identify and compare how many TFs have their motif present in each oligo

sky_df <- data.frame()
for(celltype in unique(tf_plots$celltype)){
  message(celltype)
  cell_ID_split <- colsplit(standard_res[[celltype]]$ID, "\\^", c("enhancer","silencer"))
  for(enh in unique(tf_plots$enhancer)){
    message(enh)
    for(tf in colnames(chip_tf_celltypes[[celltype]])[5:ncol(chip_tf_celltypes[[celltype]])]){
      bound_l2fc <- tf_plots$log2FoldChange[tf_plots$silencer %in% chip_tf_celltypes[[celltype]]$name[which(chip_tf_celltypes[[celltype]][,colnames(chip_tf_celltypes[[celltype]])==tf]==1)] & tf_plots$celltype==celltype & tf_plots$enhancer==enh]
      unbound_l2fc <- tf_plots$log2FoldChange[tf_plots$silencer %in% chip_tf_celltypes[[celltype]]$name[which(chip_tf_celltypes[[celltype]][,colnames(chip_tf_celltypes[[celltype]])==tf]==0)] & tf_plots$celltype==celltype & tf_plots$enhancer==enh]
      bound_med <- median(bound_l2fc, na.rm = T)
      unbound_med <- median(unbound_l2fc, na.rm = T)
      if(sum(is.na(bound_l2fc))==length(bound_l2fc)){
        bound_l2fc <- c()
      }
      if(sum(is.na(unbound_l2fc))==length(unbound_l2fc)){
        unbound_l2fc <- c()
      }
      if(length(bound_l2fc) > 0 & length(unbound_l2fc) > 0){
        p_value <- wilcox.test(bound_l2fc,unbound_l2fc)$p.value
      }
      if(length(bound_l2fc) == 0 | length(unbound_l2fc) ==0){
        p_value <- NA
      }
      percent_bound <- sum(chip_tf_celltypes[[celltype]][,colnames(chip_tf_celltypes[[celltype]])==tf]==1)/nrow(chip_tf_celltypes[[celltype]])
      temp <- data.frame(tf,celltype,enh,p_value,bound_med,unbound_med,percent_bound)
      sky_df <- rbind(sky_df, temp)
    }
  }
}

sky_df <- sky_df[order(sky_df$p_value, decreasing = T),]
rownames(sky_df) <- seq(1:nrow(sky_df))

sky_df$tf <- factor(sky_df$tf, levels = sky_df$tf[order(sky_df$p_value, decreasing = T)])

for(celltype in unique(sky_df$celltype)){
  for(enh in unique(sky_df$enh)){
    sub_df <- sky_df[which(sky_df$celltype==celltype & sky_df$enh==enh),]
    sub_df$tf <- factor(sub_df$tf, levels = sub_df$tf[order(complete.cases(sub_df$p_value), decreasing = T)])
    png(paste0("plots/tf_analysis/skyscraper_plots/neg_log10_p/tf_names_",celltype,"_",enh,"_neglog10p.png"), width = 1620, height = 1620)
    print(ggplot(sub_df, aes(x=Target, y=-log10(p_value))) + geom_point() + labs(title = paste0("skyscraper plot: ",celltype," - ",enh,"\n-log10(p)")) + theme_light() + theme(axis.text.x = element_text(angle = 90)))
    dev.off()
  }
}
## Delta median of bound - unbound, color if MWU test has a significant p-value. Taking median of the l2FC for the delta median calculation

sky_df$significance <- "> 0.05"
sky_df$significance[which(sky_df$p_value < 0.05)] <- "< 0.05" 

sky_df$significance <- factor(sky_df$significance, levels = c("< 0.05","> 0.05"))

sky_df$delta_med <- sky_df$bound_med - sky_df$unbound_med

accession_tf_name_map <- read.delim("ENCODE_ChIP/complete_histone_tf_list.txt", stringsAsFactors = F)
colnames(accession_tf_name_map) <- c("Accession","Target","Celltype")

sky_df <- merge(sky_df,accession_tf_name_map[,1:2], by.x="tf",by.y="Accession", all=T)

bind_num_tf_plots <- tf_plots[,1:5]

bound_tf_names <- list()
bind_num_tf_plots$TF_num <- 0
for(celltype in names(chip_tf_celltypes)){
  bound_tf_names[[celltype]] <- list()
  message(celltype)
  for(sil in unique(bind_num_tf_plots$silencer[which(bind_num_tf_plots$celltype==celltype)])){
    acc_list <- colnames(chip_tf_celltypes[[celltype]])[chip_tf_celltypes[[celltype]][which(chip_tf_celltypes[[celltype]]$name==sil),]==1]
    tf_list <- accession_tf_name_map$Target[which(accession_tf_name_map$Accession %in% acc_list)]
    bound_tf_names[[celltype]][[sil]] <- unique(tf_list)
    total_bound_not_rest <- sum(bound_tf_names[[celltype]][[sil]]!="REST")
    bind_num_tf_plots$TF_num[which(bind_num_tf_plots$silencer==sil & bind_num_tf_plots$celltype==celltype)] <- total_bound_not_rest
  }
}

bound_tf_names_filt <- list()
bind_num_tf_plots$TF_num_filt <- 0
for(celltype in names(chip_tf_celltypes)){
  bound_tf_names_filt[[celltype]] <- list()
  message(celltype)
  for(sil in unique(bind_num_tf_plots$silencer[which(bind_num_tf_plots$celltype==celltype)])){
    acc_list <- colnames(chip_tf_celltypes[[celltype]])[chip_tf_celltypes[[celltype]][which(chip_tf_celltypes[[celltype]]$name==sil),]==1]
    acc_list <- acc_list[acc_list %in% sky_df$tf[sky_df$percent_bound>0.1]]
    tf_list <- accession_tf_name_map$Target[which(accession_tf_name_map$Accession %in% acc_list)]
    bound_tf_names_filt[[celltype]][[sil]] <- unique(tf_list)
    total_bound_not_rest <- sum(bound_tf_names_filt[[celltype]][[sil]]!="REST")
    bind_num_tf_plots$TF_num_filt[which(bind_num_tf_plots$silencer==sil & bind_num_tf_plots$celltype==celltype)] <- total_bound_not_rest
  }
}

freq_table_filt <- list()
for(celltype in unique(bind_num_tf_plots$celltype)){
  message(celltype)
  freq_table_filt[[celltype]] <- table(bind_num_tf_plots$TF_num_filt[which(bind_num_tf_plots$celltype==celltype)])
}


## Making the plots
head(bind_num_tf_plots)

bins12 <- c()
for(i in 1:12){
  bins12 <- c(bins12, rgb(red = 30/255, green = 144/255, blue = 255/255, alpha = i/12))
}

for(celltype in unique(bind_num_tf_plots$celltype)){
  message(celltype)
  neg_ctrls_temp <- standard_res[[celltype]][which(standard_res[[celltype]]$project=="NegCtrl"),c("ID","log2FoldChange")]
  neg_ctrls_temp_split <- colsplit(neg_ctrls_temp$ID, "\\^", c("enhancer","silencer"))
  neg_ctrls_temp$celltype <- celltype
  neg_ctrls_temp$enhancer <- neg_ctrls_temp_split$enhancer
  neg_ctrls_temp$silencer <- neg_ctrls_temp_split$silencer
  neg_ctrls_temp$TF_num <- "NegCtrl"
  neg_ctrls_temp$TF_num_filt <- "NegCtrl"
  for(enh in unique(bind_num_tf_plots$enhancer)){
    message(enh)
    neg_ctrls_enh <- neg_ctrls_temp[which(neg_ctrls_temp$enhancer==enh),]
    plot_temp <- rbind(bind_num_tf_plots[which(bind_num_tf_plots$celltype==celltype & bind_num_tf_plots$enhancer==enh),], neg_ctrls_enh)
    plot_temp$non_rest_tf <- plot_temp$TF_num
    plot_temp$non_rest_tf[which(as.numeric(plot_temp$TF_num) >5 & as.numeric(plot_temp$TF_num) <11)] <- "6-10"
    plot_temp$non_rest_tf[which(as.numeric(plot_temp$TF_num) >10 & as.numeric(plot_temp$TF_num) <21)] <- "11-20"
    plot_temp$non_rest_tf[which(as.numeric(plot_temp$TF_num) >20 & as.numeric(plot_temp$TF_num) <31)] <- "21-30"
    plot_temp$non_rest_tf[which(as.numeric(plot_temp$TF_num) >30 & as.numeric(plot_temp$TF_num) <41)] <- "31-40"
    plot_temp$non_rest_tf[which(as.numeric(plot_temp$TF_num) >40 & as.numeric(plot_temp$TF_num) <51)] <- "41-50"
    plot_temp$non_rest_tf[which(as.numeric(plot_temp$TF_num) >50)] <- "> 50"
    plot_temp$non_rest_tf_filt <- plot_temp$TF_num_filt
    plot_temp$non_rest_tf_filt[which(as.numeric(plot_temp$TF_num_filt) >5 & as.numeric(plot_temp$TF_num_filt) <11)] <- "6-10"
    plot_temp$non_rest_tf_filt[which(as.numeric(plot_temp$TF_num_filt) >10 & as.numeric(plot_temp$TF_num_filt) <21)] <- "11-20"
    plot_temp$non_rest_tf_filt[which(as.numeric(plot_temp$TF_num_filt) >20 & as.numeric(plot_temp$TF_num_filt) <31)] <- "21-30"
    plot_temp$non_rest_tf_filt[which(as.numeric(plot_temp$TF_num_filt) >30 & as.numeric(plot_temp$TF_num_filt) <41)] <- "31-40"
    plot_temp$non_rest_tf_filt[which(as.numeric(plot_temp$TF_num_filt) >40 & as.numeric(plot_temp$TF_num_filt) <51)] <- "41-50"
    plot_temp$non_rest_tf_filt[which(as.numeric(plot_temp$TF_num_filt) >50)] <- "> 50"
    plot_temp$non_rest_tf <- factor(plot_temp$non_rest_tf, levels = c(0,1,2,3,4,5,"6-10","11-20","21-30","31-40","41-50","> 50"))
    plot_temp$non_rest_tf_filt <- factor(plot_temp$non_rest_tf_filt, levels = c(0,1,2,3,4,5,"6-10","11-20","21-30","31-40","41-50","> 50"))
    plot_temp$project <- "motif"
    plot_temp$project[plot_temp$TF_num=="NegCtrl"] <- "NegCtrl"
    plot_temp$project <- factor(plot_temp$project, levels = c("NegCtrl","motif"))
    # pdf(paste0("plots/tf_analysis/box_plots/number_tf_bound/no_filter/l2fc_neg_ctrls_v_motif_",celltype,"_",enh,"_blue.pdf"), width = 3.55, height = 3.55)
    # print(ggplot(plot_temp, aes(x=project, y=log2FoldChange, group=interaction(project,non_rest_tf))) + geom_boxplot(aes(fill=non_rest_tf),lwd=.24) + labs(title = paste0("log2FoldChange NegCtrl vs Motif - Number of Non-REST Bound Transcription Factors\n", celltype," - ",enh)) + theme_light(base_size = 6) + scale_fill_manual(values = bins12, name="# of\nTFs"))
    # dev.off()
    pdf(paste0("plots/tf_analysis/box_plots/number_tf_bound/filtered/l2fc_neg_ctrls_v_motif_",celltype,"_",enh,"_blue.pdf"), width = 3.55, height = 3.55/.8)
    print(ggplot(plot_temp, aes(x=project, y=log2FoldChange, group=interaction(project,non_rest_tf_filt))) +
            geom_boxplot(aes(fill=non_rest_tf_filt), lwd=.24, outlier.shape = NA) +
            labs(title = paste0("log2FC NegCtrl vs Motif - # of Non-REST Bound TFs\n", celltype," - ",enh,"\tFiltered by Percent Bound")) +
            theme_light(base_size = 6) + theme(legend.position = "bottom",legend.margin=margin(t = 0, unit='cm'),legend.justification = "left") +
            scale_fill_manual(values = bins12, name="# of\nTFs", na.translate=T, na.value="grey") + guides(fill=guide_legend(nrow = 2)))
    dev.off()
  }
}

new_tf_num_plot_df <- for_rodrigo[,1:9]
new_tf_num_plot_df$TF_num <- NA
new_tf_num_plot_df$TF_A_B <- NA
new_tf_num_plot_df$TF_C_B <- NA
for(celltype in unique(new_tf_num_plot_df$celltype)){
  message(celltype)
  for(sil in unique(new_tf_num_plot_df$silencer[which(new_tf_num_plot_df$celltype==celltype)])){
    if(new_tf_num_plot_df$project[which(new_tf_num_plot_df$silencer==sil & new_tf_num_plot_df$celltype==celltype)][1]=="NegCtrl") next
    total_bound_not_rest <- sum(bound_tf_names[[celltype]][[sil]]!="REST")
    new_tf_num_plot_df$TF_num[which(new_tf_num_plot_df$celltype==celltype & new_tf_num_plot_df$silencer==sil)] <-total_bound_not_rest
    for(enh in unique(new_tf_num_plot_df$enhancer)){
      total_bound_not_rest_A_B <- sum(bound_tf_names[[celltype]][[sil]]!="REST" & bound_tf_names[[celltype]][[sil]] %in% med_comp_plot_df$tf[which(med_comp_plot_df$celltype==celltype & med_comp_plot_df$enh==enh & med_comp_plot_df$significance=="both" & med_comp_plot_df$quadrant==1)])
      total_bound_not_rest_C_B <- sum(bound_tf_names[[celltype]][[sil]]!="REST" & bound_tf_names[[celltype]][[sil]] %in% med_comp_plot_df$tf[which(med_comp_plot_df$celltype==celltype & med_comp_plot_df$enh==enh & med_comp_plot_df$significance=="both" & med_comp_plot_df$quadrant==2)])
      new_tf_num_plot_df$TF_A_B[which(new_tf_num_plot_df$celltype==celltype & new_tf_num_plot_df$silencer==sil & new_tf_num_plot_df$enhancer==enh)] <-total_bound_not_rest_A_B
      new_tf_num_plot_df$TF_C_B[which(new_tf_num_plot_df$celltype==celltype & new_tf_num_plot_df$silencer==sil & new_tf_num_plot_df$enhancer==enh)] <-total_bound_not_rest_C_B
    }
  }
}

new_tf_num_plot_df$tf_bins <- new_tf_num_plot_df$TF_num
new_tf_num_plot_df$tf_bins[which(as.numeric(new_tf_num_plot_df$TF_num) > 5 & as.numeric(new_tf_num_plot_df$TF_num) < 11)] <- "6-10"
new_tf_num_plot_df$tf_bins[which(as.numeric(new_tf_num_plot_df$TF_num) > 10 & as.numeric(new_tf_num_plot_df$TF_num) < 21)] <- "11-20"
new_tf_num_plot_df$tf_bins[which(as.numeric(new_tf_num_plot_df$TF_num) > 20 & as.numeric(new_tf_num_plot_df$TF_num) < 31)] <- "21-30"
new_tf_num_plot_df$tf_bins[which(as.numeric(new_tf_num_plot_df$TF_num) > 30 & as.numeric(new_tf_num_plot_df$TF_num) < 41)] <- "31-40"
new_tf_num_plot_df$tf_bins[which(as.numeric(new_tf_num_plot_df$TF_num) > 40 & as.numeric(new_tf_num_plot_df$TF_num) < 51)] <- "41-50"
new_tf_num_plot_df$tf_bins[which(new_tf_num_plot_df$TF_num > 50)] <- ">50"

new_tf_num_plot_df$tf_bins <- factor(new_tf_num_plot_df$tf_bins, levels = c("NegCtrl",0,1,2,3,4,5,"6-10","11-20","21-30","31-40","41-50",">50"))
new_tf_num_plot_df$tf_bins[is.na(new_tf_num_plot_df$tf_bins)] <- "NegCtrl"

new_tf_num_plot_df$tf_bins_A_B <- new_tf_num_plot_df$TF_A_B
new_tf_num_plot_df$tf_bins_A_B[which(as.numeric(new_tf_num_plot_df$TF_A_B) > 5 & as.numeric(new_tf_num_plot_df$TF_A_B) < 11)] <- "6-10"
new_tf_num_plot_df$tf_bins_A_B[which(as.numeric(new_tf_num_plot_df$TF_A_B) > 10 & as.numeric(new_tf_num_plot_df$TF_A_B) < 21)] <- "11-20"
new_tf_num_plot_df$tf_bins_A_B[which(as.numeric(new_tf_num_plot_df$TF_A_B) > 20 & as.numeric(new_tf_num_plot_df$TF_A_B) < 31)] <- "21-30"
new_tf_num_plot_df$tf_bins_A_B[which(as.numeric(new_tf_num_plot_df$TF_A_B) > 30 & as.numeric(new_tf_num_plot_df$TF_A_B) < 41)] <- "31-40"
new_tf_num_plot_df$tf_bins_A_B[which(as.numeric(new_tf_num_plot_df$TF_A_B) > 40 & as.numeric(new_tf_num_plot_df$TF_A_B) < 51)] <- "41-50"
new_tf_num_plot_df$tf_bins_A_B[which(new_tf_num_plot_df$TF_A_B > 50)] <- ">50"

new_tf_num_plot_df$tf_bins_A_B <- factor(new_tf_num_plot_df$tf_bins_A_B, levels = c("NegCtrl",0,1,2,3,4,5,"6-10","11-20","21-30","31-40","41-50",">50"))
new_tf_num_plot_df$tf_bins_A_B[which(new_tf_num_plot_df$project=="NegCtrl")] <- "NegCtrl"

new_tf_num_plot_df$tf_bins_C_B <- new_tf_num_plot_df$TF_C_B
new_tf_num_plot_df$tf_bins_C_B[which(as.numeric(new_tf_num_plot_df$TF_C_B) > 5 & as.numeric(new_tf_num_plot_df$TF_C_B) < 11)] <- "6-10"
new_tf_num_plot_df$tf_bins_C_B[which(as.numeric(new_tf_num_plot_df$TF_C_B) > 10 & as.numeric(new_tf_num_plot_df$TF_C_B) < 16)] <- "11-15"
new_tf_num_plot_df$tf_bins_C_B[which(as.numeric(new_tf_num_plot_df$TF_C_B) > 15)] <- ">15"

new_tf_num_plot_df$tf_bins_C_B <- factor(new_tf_num_plot_df$tf_bins_C_B, levels = c("NegCtrl",0,1,2,3,4,5,"6-10","11-15",">15"))
new_tf_num_plot_df$tf_bins_C_B[which(new_tf_num_plot_df$project=="NegCtrl")] <- "NegCtrl"



bins_group <- data.frame()
for(celltype in unique(new_tf_num_plot_df$celltype)){
  message(celltype)
  for(enh in unique(new_tf_num_plot_df$enhancer)){
    message(enh)
    cell_enh_sub <- new_tf_num_plot_df[which(new_tf_num_plot_df$enhancer==enh & new_tf_num_plot_df$celltype==celltype),]
    neg_ctrls_enh <- cell_enh_sub[which(cell_enh_sub$project=="NegCtrl"),]
    neg_ctrls_med <- median(neg_ctrls_enh$log2FoldChange, na.rm = T)
    for(quad in names(new_num_tf_plot_info)){
      message(quad)
      plot_temp <- cell_enh_sub[which(cell_enh_sub$silencer %in% new_num_tf_plot_info[[quad]][[celltype]][[enh]]),]
      if(nrow(plot_temp) == 0) next
      if(quad=="1"){
        group <- "A_B"
        # skew_plot <- ggplot(plot_temp, aes(x=tf_bins_A_B, y=LogSkew, fill=tf_bins_A_B)) + geom_boxplot(outlier.shape = NA) + scale_fill_manual(values=bins12) + theme_light() +
        #   theme(legend.position = "none") +xlab("Number of TFs")
        # skew_line <- ggplot(plot_temp, aes(x=tf_bins_A_B, y=LogSkew, fill=tf_bins_A_B)) + geom_hline(yintercept = 0, col="red") + geom_boxplot(outlier.shape = NA) + scale_fill_manual(values=bins12) + theme_light() +
        #   theme(legend.position = "none") +xlab("Number of TFs")
        # bound_stats <- data.frame(table(plot_temp[,c("tf_bins_A_B","REST")]))
      }
      if(quad=="2"){
        group <- "C_B"
        # skew_plot <- ggplot(plot_temp, aes(x=tf_bins_C_B, y=LogSkew, fill=tf_bins_C_B)) + geom_boxplot(outlier.shape = NA) + scale_fill_manual(values=bins12) + theme_light() +
        #   theme(legend.position = "none") +xlab("Number of TFs")
        # skew_line <- ggplot(plot_temp, aes(x=tf_bins_C_B, y=LogSkew, fill=tf_bins_C_B)) + geom_hline(yintercept = 0, col="red") + geom_boxplot(outlier.shape = NA) + scale_fill_manual(values=bins12) + theme_light() +
        #   theme(legend.position = "none") +xlab("Number of TFs")
        # bound_stats <- data.frame(table(plot_temp[,c("tf_bins_C_B","REST")]))
      }
      
      # bound_stats$percent <- NA
      # for(i in 1:nrow(bound_stats)){
      #   bin_sum <- sum(bound_stats$Freq[which(bound_stats[,1]==bound_stats[i,1])])
      #   if(bin_sum==0) next
      #   else{
      #     bound_stats$percent[i] <- round(bound_stats$Freq[i]/bin_sum,digits = 4)
      #   }
      # }
      # colnames(bound_stats) <- c(paste0("tf_bins_",group),"REST","Freq","percent")
      # if(quad=="1"){
      #   bound_plot <- ggplot(bound_stats[complete.cases(bound_stats),], aes(x=tf_bins_A_B, y=Freq, fill=REST)) + geom_bar(stat="identity") + theme_light(base_size = 6) +
      #   geom_text(aes(label = percent), position = position_stack(vjust = 0.5), size=2) + scale_fill_manual(values = brewer_set2[1:3]) + theme(legend.position = "bottom",legend.box = "vertical",legend.margin=margin(t = 0, unit='cm'), 
      #                                                                                                                                  panel.grid = element_line(size = 0.24),legend.key.height = unit(2,'mm'), legend.key.width = unit(4,'mm'), legend.text.align = 0) +
      #   xlab("Number of TFs") + guides(fill=guide_legend(nrow = 1), title="REST binding") + ylab("Number of Oligos")
      # }
      # if(quad=="2"){
      #   bound_plot <- ggplot(bound_stats[complete.cases(bound_stats),], aes(x=tf_bins_C_B, y=Freq, fill=REST)) + geom_bar(stat="identity") + theme_light(base_size = 6) +
      #     geom_text(aes(label = percent), position = position_stack(vjust = 0.5), size=2) + scale_fill_manual(values = brewer_set2[1:3]) + theme(legend.position = "bottom",legend.box = "vertical",legend.margin=margin(t = 0, unit='cm'), 
      #                                                                                                                                            panel.grid = element_line(size = 0.24),legend.key.height = unit(2,'mm'), legend.key.width = unit(4,'mm'), legend.text.align = 0) +
      #     xlab("Number of TFs") + guides(fill=guide_legend(nrow = 1), title="REST binding") + ylab("Number of Oligos")
      # }
      plot_tmp2 <- rbind(neg_ctrls_enh, plot_temp)
      if(quad=="1"){
        fc_plot <- ggplot(plot_tmp2, aes(x=tf_bins_A_B, y=log2FoldChange, fill=tf_bins_A_B)) + geom_hline(yintercept = neg_ctrls_med, col="red") + geom_boxplot(outlier.shape = NA) + scale_fill_manual(values=c("grey",bins12)) +
          theme_light() + theme(legend.position = "none")  + xlab("Number of TFs")
      }
      if(quad=="2"){
        fc_plot <- ggplot(plot_tmp2, aes(x=tf_bins_C_B, y=log2FoldChange, fill=tf_bins_C_B)) + geom_hline(yintercept = neg_ctrls_med, col="red") + geom_boxplot(outlier.shape = NA) + scale_fill_manual(values=c("grey",bins12)) +
          theme_light() + theme(legend.position = "none")  + xlab("Number of TFs")
      }
      if(celltype=="GM12878" & enh=="En02" & quad=="1"){
        plot_tmp2$group <- group
        bins_group <- plot_tmp2
      }
      else{
        if(nrow(plot_temp)>0){
          plot_tmp2$group <- group
          bins_group <- rbind(bins_group, plot_tmp2)
        }
      }
      # bound_stats <- bound_stats[complete.cases(bound_stats),]
      # if(quad=="1"){
      #   bound_skew_plot <- ggplot(bound_stats[which(bound_stats$REST==1),], aes(x=tf_bins_A_B, y=percent, col=brewer_set2[2], group=1)) + geom_point() + scale_y_continuous(position = "right", limits = c(0,1), name="Fraction REST Bound") +
      #     geom_line() + theme_light(base_size = 6) + theme(legend.position = "none", axis.title.y.right = element_text(color = brewer_set2[2])) + xlab("Number of TFs") 
      # }
      # if(quad=="2"){
      #   bound_skew_plot <- ggplot(bound_stats[which(bound_stats$REST==1),], aes(x=tf_bins_C_B, y=percent, col=brewer_set2[2], group=1)) + geom_point() + scale_y_continuous(position = "right", limits = c(0,1), name="Fraction REST Bound") +
      #     geom_line() + theme_light(base_size = 6) + theme(legend.position = "none", axis.title.y.right = element_text(color = brewer_set2[2])) + xlab("Number of TFs") 
      # }
      # message("plotting skew")
      # pdf(paste0("plots/tf_analysis/box_plots/skew_",group,"_",enh,"_",celltype,".pdf"), width=170/25.4, height = 50/25.4)
      # print(skew_plot)
      # dev.off()
      # message("plotting percent")
      # pdf(paste0("plots/tf_analysis/bar_plots/percent_bound_",group,"_",enh,"_",celltype,".pdf"), width=170/25.4, height = 40/25.4)
      # print(bound_plot)
      # dev.off()
      message("plotting l2fc")
      pdf(paste0("plots/tf_analysis/box_plots/l2fc_square_",group,"_",enh,"_",celltype,".pdf"), width=40/25.4, height = 40/25.4)
      print(fc_plot)
      dev.off()
      # message("plotting skew 2.0")
      # pdf(paste0("plots/tf_analysis/overlap_plots/skew_line_",group,"_",enh,"_",celltype,".pdf"), width=40/25.4, height = 40/25.4)
      # print(skew_line + theme_light(base_size = 6) + theme(axis.title.y = element_text(color = "dodgerblue"), legend.position = "none"))
      # dev.off()
      # message("plotting skew bound")
      # pdf(paste0("plots/tf_analysis/overlap_plots/bound_skew_",group,"_",enh,"_",celltype,".pdf"), width = 40/25.4, height = 40/25.4)
      # print(bound_skew_plot)
      # dev.off()
    }
  }
}

pval_group_list <- list()
for(celltype in unique(bins_group$celltype)){
  message(celltype)
  pval_group_list[[celltype]] <- list()
  for(enh in unique(bins_group$enhancer)){
    message(enh)
    pval_group_list[[celltype]][[enh]] <- list()
    for(group in unique(bins_group$group)){
      message(group)
      group_enh_cell_sub <- bins_group[which(bins_group$celltype==celltype & bins_group$enhancer==enh & bins_group$group==group),]
      if(nrow(group_enh_cell_sub)==0) next
      pval_group_list[[celltype]][[enh]][[group]] <- list()
      group_enh_cell_sub <- remove.factors(group_enh_cell_sub)
      if(group=="A_B"){
        message("l2fc")
        pvals_l2fc <- pairwise_wilcox_test(group_enh_cell_sub, log2FoldChange ~ tf_bins_A_B, ref.group = "NegCtrl", p.adjust.method = "BH")
        message("skew")
        group_enh_cell_sub <- group_enh_cell_sub[complete.cases(group_enh_cell_sub$LogSkew),]
        pvals_skew <- pairwise_wilcox_test(group_enh_cell_sub, LogSkew ~ tf_bins_A_B, ref.group = "1", p.adjust.method = "BH")
      }
      if(group=="C_B"){
        message("l2fc")
        groups <- as.data.frame(table(group_enh_cell_sub$tf_bins_C_B))
        pvals_l2fc <- pairwise_wilcox_test(group_enh_cell_sub, log2FoldChange ~ tf_bins_C_B, ref.group = "NegCtrl", p.adjust.method = "BH")
        message("skew")
        group_enh_cell_sub <- group_enh_cell_sub[complete.cases(group_enh_cell_sub$LogSkew),]
        pvals_skew <- pairwise_wilcox_test(group_enh_cell_sub, LogSkew ~ tf_bins_C_B, ref.group = "1", p.adjust.method = "BH")
      }
      pval_group_list[[celltype]][[enh]][[group]]$l2fc <- pvals_l2fc
      pval_group_list[[celltype]][[enh]][[group]]$skew <- pvals_skew
    }
  }
}

In [ ]:
### Figure F-I
## Generate box plots to visualize the differences between Negative Controls and combinations of REST/MIER1/EHMT2
## and REST/RCOR1/HDAC2 binding in oligos

neg_ctrl_all <- data.frame()
for(celltype in names(standard_res)){
  message(celltype)
  neg_ctrl_temp <- standard_res[[celltype]][which(standard_res[[celltype]]$project=="NegCtrl"),c("ID","log2FoldChange","project")]
  neg_ctrl_temp$celltype <- celltype
  neg_ctrl_temp$scr_l2fc <- NA
  neg_ctrl_temp$LogSkew <- NA
  neg_ctrl_temp$REST <- "NegCtrl"
  neg_ctrl_all <- rbind(neg_ctrl_all, neg_ctrl_temp)
}

neg_id_split <- colsplit(neg_ctrl_all$ID,"\\^",c("enhancer","silencer"))
neg_ctrl_all$enhancer <- neg_id_split$enhancer
neg_ctrl_all$silencer <- neg_id_split$silencer
neg_ctrl_all$REST <- 0

bound_factor <- merge(bound_factor, RESTscreen_attr_comb[,c("ID","project")], by="ID", all.x=T)
fig3 <- bound_factor[,c("ID","enhancer","silencer","celltype","log2FoldChange","scr_l2fc","LogSkew","project","REST")]
fig3 <- rbind(neg_ctrl_all, fig3)

## Set up motif positivity for use in plots

fig3$RCOR1 <- "-"
fig3$REST <- "-"
fig3$HDAC2 <- "-"
fig3$MIER <- "-"
fig3$EHMT2 <- "-"
for(celltype in unique(fig3$celltype)){
  message(celltype)
  fig3$REST[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$REST)] <- "+"
  fig3$RCOR1[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$RCOR1)] <- "+"
  fig3$HDAC2[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$HDAC2)] <- "+"
  fig3$MIER[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$MIER1)] <- "+"
  fig3$EHMT2[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$EHMT2)] <- "+"
}

fig3$all <- interaction(fig3$REST,fig3$MIER,fig3$EHMT2,fig3$RCOR1,fig3$HDAC2, sep="/")

fig3$REST_RCOR_HDAC <- interaction(fig3$REST,fig3$RCOR1,fig3$HDAC2, sep = "/")
fig3$REST_MIER_EHMT2 <- interaction(fig3$REST,fig3$MIER,fig3$EHMT2, sep = "/")

fig3 <- remove.factors(fig3)
fig3$all[which(fig3$project=="NegCtrl")] <- "NegCtrl"
fig3$REST_MIER_EHMT2[which(fig3$project=="NegCtrl")] <- "NegCtrl"
fig3$REST_RCOR_HDAC[which(fig3$project=="NegCtrl")] <- "NegCtrl"

## Run plots and get statistics through a Wilcox Test comparing the significance between each motif positivity group and the negative controls or REST/TF1/TF2 negative for skew

# inter_stats <- list()
for(celltype in c("K562")){
  message(celltype)
  # inter_stats[[celltype]] <- list()
  for(enh in c("En19")){
    message(enh)
    # inter_stats[[celltype]][[enh]] <- list()
    plot_temp <- fig3[which(fig3$enhancer==enh & fig3$celltype==celltype),]
    if(nrow(plot_temp[which(plot_temp$project=="motif"),])==0) next
    
    ## Cycle through the sets that I need to plot along the x-axis and subset each plot_temp and skew_temp to include only that
    ## Rename the columns to have the relevant column include only the immediately relevant columns: celltype, enhancer, silencer, ID, log2FoldChange, LogSkew and relevant binding
    ## re-name columns to normalize the name of the binding column across all different sets.
    ## Make sure to get the names to match up properly for xlab and legend title
    
    # for(inter in c("REST_RCOR_HDAC", "REST_MIER_EHMT2")){
    # message(inter)
    # skew_levels <- c("-/-/-","+/-/-","+/+/-","+/-/+","+/+/+","-/+/-","-/-/+","-/+/+")
    # l2fc_levels <- c("NegCtrl","+/-/-","+/+/-","+/-/+","+/+/+","-/-/-","-/+/-","-/-/+","-/+/+")
    
    # if(inter=="REST_RCOR_HDAC"){
    #   lab_name <- "REST/RCOR/HDAC binding"
    # }
    # if(inter=="REST_MIER_EHMT2"){
    #   lab_name <- "REST/MIER1/EHMT2"
    # }
    
    
    l2fc_levels <- c("NegCtrl","+/-/-/-/-","+/+/-/-/-","+/-/+/-/-","+/-/-/+/-","+/-/-/-/+","+/+/+/-/-","+/+/-/+/-","+/+/-/-/+","+/-/+/+/-","+/-/+/-/+","+/-/-/+/+","+/-/+/+/+","+/+/-/+/+","+/+/+/-/+","+/+/+/+/-","+/+/+/+/+",
                     "-/-/-/-/-","-/+/-/-/-","-/-/+/-/-","-/-/-/+/-","-/-/-/-/+","-/+/+/-/-","-/+/-/+/-","-/+/-/-/+","-/-/+/+/-","-/-/+/-/+","-/-/-/+/+","-/-/+/+/+","-/+/+/-/+","-/+/-/+/+","-/+/+/+/-","-/+/+/+/+")
    lab_name <- "REST/MIER/EHMT2/RCOR/HDAC2"
    
    red_plot <- plot_temp[,c("ID", "enhancer","silencer", "celltype", "log2FoldChange", "scr_l2fc", "LogSkew", "project","all")] #inter instead of all if redoing things
    colnames(red_plot) <- c(colnames(red_plot)[1:8],"binding")
    # skew_temp <- red_plot[complete.cases(red_plot$LogSkew),]
    
    red_plot$binding <- factor(red_plot$binding, levels = l2fc_levels)
    
    # skew_temp$binding <- factor(skew_temp$binding, levels = skew_levels)
    
    l2fc_min <- -2.5
    l2fc_max <- 10.5
    # skew_min <- -3
    # skew_max <- 5.5
    
    neg_ctrl_med <- median(red_plot$log2FoldChange[which(red_plot$project=="NegCtrl")], na.rm = T)
    
    pdf(paste0("plots/tf_analysis/box_plots/neg_ctrl_motif_comp/",enh,"_",celltype,"_all_yrange.pdf"), width=10, height = 5) #width=40/25.4, height=45/25.4)
    print(ggplot(red_plot, aes(x=binding,y=log2FoldChange, fill=binding)) + geom_hline(yintercept = neg_ctrl_med,lwd=0.2) + geom_boxplot(outlier.shape = NA,lwd=0.2) + theme_light(base_size = 6) +
            coord_cartesian(ylim = c(l2fc_min,l2fc_max)) + xlab(lab_name) + # scale_fill_manual(values=c("grey",brewer_paired[c(1,3,5,7,2,4,6,8)])) + xlab(lab_name) +
            theme(legend.position = "none",axis.text.x = element_text(angle = 45, hjust = 1)))
    dev.off()
    # pdf(paste0("plots/tf_analysis/box_plots/skew/",enh,"_",celltype,"_",inter,"_yrange.pdf"), width=40/25.4, height=45/25.4)
    # print(ggplot(skew_temp, aes(x=binding,y=LogSkew, fill=binding)) + geom_hline(yintercept = 0,lwd=0.2) + geom_boxplot(outlier.shape = NA, lwd=0.2) + theme_light(base_size = 6) +
    #         coord_cartesian(ylim = c(skew_min,skew_max)) + scale_fill_manual(values=c(brewer_paired[c(2,1,3,5,7,4,6,8)])) + xlab(lab_name) +
    #         theme(legend.position = "none",axis.text.x = element_text(angle = 45, hjust = 1)))
    # dev.off()
    
    # }
    
    # message("l2fc sig")
    # inter_stats[[celltype]][[enh]][[inter]]$l2fc <- red_plot %>%
    #   wilcox_test(formula = log2FoldChange ~ binding, ref.group = "NegCtrl") %>%
    #   adjust_pvalue(method = "BH") %>%
    #   add_significance()
    # 
    # message("skew sig")
    # if(inter=="CoREST"){
    #   inter_stats[[celltype]][[enh]][[inter]]$skew <- skew_temp %>%
    #     wilcox_test(formula = LogSkew ~ binding, ref.group = "-/-") %>%
    #     adjust_pvalue(method = "BH") %>%
    #     add_significance()
    # }
    # if(inter!="CoREST"){
    #   inter_stats[[celltype]][[enh]][[inter]]$skew <- skew_temp %>%
    #     wilcox_test(formula = LogSkew ~ binding, ref.group = "-/-/-") %>%
    #     adjust_pvalue(method = "BH") %>%
    #     add_significance()
    # }
  }
}

fig3$TRIP13 <- "-"
fig3$PTTG1 <- "-"
for(celltype in unique(fig3$celltype)){
  message(celltype)
  fig3$TRIP13[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$TRIP13)] <- "+"
  fig3$PTTG1[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$PTTG1)] <- "+"
  
}

fig3$score <- "NoScore"
for(celltype in unique(fig3$celltype)){
  message(celltype)
  for(enh in enh_all){
    message(enh)
    for(relat in names(oligo_split_list[[celltype]][[enh]])){
      fig3$score[which(fig3$silencer %in% oligo_split_list[[celltype]][[enh]][[relat]] & 
                         fig3$celltype==celltype & fig3$enhancer==enh)] <- relat
    }
  }
}

fig3$score[which(fig3$score=="NoScore" & fig3$project=="motif")] <- "below"



for(celltype in unique(fig3$celltype)){
  message(celltype)
  for(enh in enh_all){
    message(enh)
    for(rel in c("above","below")){
      message(rel)
      plot_temp <- fig3[which(fig3$celltype==celltype & fig3$enhancer==enh & fig3$score %in% c("NoScore",rel)),]
      message(nrow(plot_temp))
      for(inter in c("REST_RCOR_HDAC", "REST_MIER_EHMT2")){
        message(inter)
        skew_levels <- c("-/-/-","+/-/-","+/+/-","+/-/+","+/+/+","-/+/-","-/-/+","-/+/+")
        l2fc_levels <- c("NegCtrl","+/-/-","+/+/-","+/-/+","+/+/+","-/-/-","-/+/-","-/-/+","-/+/+")
        
        if(inter=="REST_RCOR_HDAC"){
          lab_name <- "REST/RCOR/HDAC binding"
        }
        if(inter=="REST_MIER_EHMT2"){
          lab_name <- "REST/MIER1/EHMT2"
        }
        
        red_plot <- plot_temp[,c("ID", "enhancer","silencer", "celltype", "log2FoldChange", "scr_l2fc", "LogSkew", "project",inter)]
        message(nrow(red_plot))
        colnames(red_plot) <- c("ID", "enhancer","silencer", "celltype", "log2FoldChange", "scr_l2fc", "LogSkew", "project","binding")
        skew_temp <- red_plot[complete.cases(red_plot$LogSkew),]
        
        red_plot$binding <- factor(red_plot$binding, levels = l2fc_levels)
        
        skew_temp$binding <- factor(skew_temp$binding, levels = skew_levels)
        
        # l2fc_min <- -2.5
        # l2fc_max <- 10.5
        # skew_min <- -3
        # skew_max <- 5.5
        
        neg_ctrl_med <- median(red_plot$log2FoldChange[which(red_plot$project=="NegCtrl")], na.rm = T)
        
        pdf(paste0("plots/tf_analysis/box_plots/neg_ctrl_motif_comp/fig_S8_above_below/",enh,"_",celltype,"_",inter,"_",rel,".pdf"), width=40/25.4, height=45/25.4)
        print(ggplot(red_plot, aes(x=binding,y=log2FoldChange, fill=binding)) + geom_hline(yintercept = neg_ctrl_med,lwd=0.2) + geom_boxplot(outlier.shape = NA,lwd=0.2) + theme_light(base_size = 6) +
                # coord_cartesian(ylim = c(l2fc_min,l2fc_max)) + 
                scale_fill_manual(values=c("grey",brewer_paired[c(1,3,5,7,2,4,6,8)])) + xlab(lab_name) +
                theme(legend.position = "none",axis.text.x = element_text(angle = 45, hjust = 1)))
        dev.off()
        pdf(paste0("plots/tf_analysis/box_plots/skew/fig_s8_above_below/",enh,"_",celltype,"_",inter,"_",rel,".pdf"), width=40/25.4, height=45/25.4)
        print(ggplot(skew_temp, aes(x=binding,y=LogSkew, fill=binding)) + geom_hline(yintercept = 0,lwd=0.2) + geom_boxplot(outlier.shape = NA, lwd=0.2) + theme_light(base_size = 6) +
                # coord_cartesian(ylim = c(skew_min,skew_max)) + 
                scale_fill_manual(values=c(brewer_paired[c(2,1,3,5,7,4,6,8)])) + xlab(lab_name) +
                theme(legend.position = "none",axis.text.x = element_text(angle = 45, hjust = 1)))
        dev.off()
        # 
      }
    }
  }
}

fig3$ZNF644 <- "-"

for(celltype in unique(fig3$celltype)){
  message(celltype)
  fig3$ZNF644[which(fig3$celltype==celltype & fig3$silencer %in% tf_sil_list[[celltype]]$ZNF644)] <- "+"
  
}

In [ ]:
###Figure 5G
## Make density plots comparing the minor allele frequency inside and outside strong motifs

emvar_comp <- data.frame()
for(celltype in names(emVAR_all)){
  keep <- emVAR_all[[celltype]][which(emVAR_all[[celltype]]$SNP %in% SNP_incl & !is.na(emVAR_all[[celltype]]$ref_allele)), c("ID","SNP","LogSkew","Skew_logFDR")]
  keep$celltype <- celltype
  emvar_comp <- rbind(emvar_comp, keep)
}

## Compile scores for ref and alt alleles into one data frame identifying which have the SNP inside and outside the center window
maf_rare_ref <- read.delim("meme-suite/maf_rare_ref_fimo.txt", stringsAsFactors = F)
maf_rare_alt <- read.delim("meme-suite/maf_rare_alt_fimo.txt", stringsAsFactors = F)

head(maf_rare_alt)
head(maf_rare_ref)

maf_rare_alt <- maf_rare_alt[order(maf_rare_alt$sequence.name),]
maf_rare_ref <- maf_rare_ref[order(maf_rare_ref$sequence.name),]

maf_rare_alt <- maf_rare_alt[maf_rare_alt$sequence.name!="",]
maf_rare_ref <- maf_rare_ref[maf_rare_ref$sequence.name!="",]

maf_rare_alt <- maf_rare_alt[which(maf_rare_alt$X.pattern.name=="MA0138.2"),]
maf_rare_ref <- maf_rare_ref[which(maf_rare_ref$X.pattern.name=="MA0138.2"),]

maf_rare_alt$loc <- "outside"
maf_rare_ref$loc <- "outside"

maf_rare_alt$loc[which(maf_rare_alt$start>90 & maf_rare_alt$stop < 112)] <- "inside"
maf_rare_ref$loc[which(maf_rare_ref$start>90 & maf_rare_ref$stop < 112)] <- "inside"

mra_split <- colsplit(maf_rare_alt$sequence.name,":",c("chr","pos","ref","alt","allele","window"))
mrr_split <- colsplit(maf_rare_ref$sequence.name,":",c("chr","pos","ref","alt","allele","window"))

mra_window <- colsplit(mra_split$window, "P", c("w","num"))
mrr_window <- colsplit(mrr_split$window, "P", c("w","num"))

maf_rare_alt$SNP <- paste(mra_split$chr,mra_split$pos,mra_split$ref,mra_split$alt, sep=":")
maf_rare_ref$SNP <- paste(mrr_split$chr,mrr_split$pos,mrr_split$ref,mrr_split$alt, sep=":")

maf_rare_alt$num <- mra_window$num
maf_rare_ref$num <- mrr_window$num

maf_rare_scores <- data.frame()
maf_rare_snps <- c(unique(maf_rare_alt$SNP), unique(maf_rare_ref$SNP))
for(snp in unique(maf_rare_snps)){
  message(snp)
  alt_temp <- maf_rare_alt[which(maf_rare_alt$SNP==snp),]
  ref_temp <- maf_rare_ref[which(maf_rare_ref$SNP==snp),]
  
  # if(nrow(alt_temp==0)){
  #   score_a <- 0
  #   loc_a <- NA
  # }
  # if(nrow(ref_temp==0)){
  #   message("no ref")
  #   score_r <- 0
  #   loc_r <- NA
  # }
  score_a <- 0
  loc_a <- NA
  score_r <- 0
  loc_r <- NA
  
  if(nrow(alt_temp)==1){
    score_a <- alt_temp$score
    loc_a <- alt_temp$loc
  }
  if(nrow(ref_temp)==1){
    score_r <- ref_temp$score
    loc_r <- ref_temp$loc
  }
  
  if(nrow(alt_temp) > 1){
    message("multiple alt")
    if("inside" %in% alt_temp$loc){
      alt_temp <- alt_temp[which(alt_temp$loc=="inside"),]
      if(nrow(alt_temp)>1){
        pos_diff <- abs(100 - unique(alt_temp$num))
        alt_temp <- alt_temp[which(pos_diff==min(pos_diff)),]
      }
      score_a <- alt_temp$score
      loc_a <- alt_temp$loc
    }
    else{
      # message("mult outside")
      loc_a <- "outside"
      score_a <- mean(alt_temp$score, na.rm=T)
    }
  }
  if(nrow(ref_temp) > 1){
    # message("multiple ref")
    if("inside" %in% ref_temp$loc){
      ref_temp <- ref_temp[which(ref_temp$loc=="inside"),]
      if(nrow(ref_temp)>1){
        pos_diff <- abs(100 - unique(ref_temp$num))
        ref_temp <- ref_temp[which(pos_diff==min(pos_diff)),]
      }
      score_r <- ref_temp$score
      loc_r <- ref_temp$loc
    }
    else{
      # message("mult outside")
      loc_r <- "outside"
      score_r <- mean(ref_temp$score, na.rm=T)
    }
  }

  a_tmp <- data.frame(snp, score_a, loc_a, score_r, loc_r)
  maf_rare_scores <- rbind(maf_rare_scores, a_tmp)
}

emvar_comp <- merge(emvar_comp[,1:10], maf_rare_scores, by.x="SNP", by.y="snp", all.x=T)

emvar_comp$DBS <- emvar_comp$score_a - emvar_comp$score_r

## Incorporate the population minor allele frequency (MAF) for each oligo and incorporate into the df

maf1_pop_af <- read.delim("fasta/maf1_AF_1KGp3_vcf.txt", header = F, sep = "\t", stringsAsFactors = F)
rare_pop_af <- read.delim("fasta/rare_AF_1KGp3_vcf.txt", header = F, sep = "\t", stringsAsFactors = F)

dim(maf1_pop_af)

colnames(maf1_pop_af) <- c("chr", "pos","rsid", "ref","alt","qual","filter","info")
colnames(rare_pop_af) <- c("chr", "pos","rsid", "ref","alt","qual","filter","info")

maf1_info_split <- colsplit(maf1_pop_af$info, "\\;", names = c("AC", "AF","AN","NS","DP","EAS","AMR","AFR","EUR","SAS","AA","VT"))
rare_info_split <- colsplit(rare_pop_af$info, "\\;", names = c("AC", "AF","AN","NS","DP","EAS","AMR","AFR","EUR","SAS","AA","VT"))
head(maf1_info_split)

maf1_EAS <- colsplit(maf1_info_split$EAS, "=", names=c("name","EAS_AF"))
maf1_EUR <- colsplit(maf1_info_split$EUR, "=", names=c("name","EUR_AF"))
maf1_AFR <- colsplit(maf1_info_split$AFR, "=", names=c("name","AFR_AF"))

rare_EAS <- colsplit(rare_info_split$EAS, "=", names=c("name","EAS_AF"))
rare_EUR <- colsplit(rare_info_split$EUR, "=", names=c("name","EUR_AF"))
rare_AFR <- colsplit(rare_info_split$AFR, "=", names=c("name","AFR_AF"))

maf1_pop_af <- cbind(maf1_pop_af[,1:7], maf1_EAS$EAS_AF, maf1_EUR$EUR_AF, maf1_AFR$AFR_AF)
rare_pop_af <- cbind(rare_pop_af[,1:7], rare_EAS$EAS_AF, rare_EUR$EUR_AF, rare_AFR$AFR_AF)

colnames(maf1_pop_af) <- c("chr", "pos","rsid", "ref","alt","qual","filter","EAS_AF", "EUR_AF","AFR_AF")
colnames(rare_pop_af) <- c("chr", "pos","rsid", "ref","alt","qual","filter","EAS_AF", "EUR_AF", "AFR_AF")


maf1_pop_af$SNP <- paste(maf1_pop_af$chr,maf1_pop_af$pos,maf1_pop_af$ref,maf1_pop_af$alt,sep = ":")
rare_pop_af$SNP <- paste(rare_pop_af$chr,rare_pop_af$pos,rare_pop_af$ref,rare_pop_af$alt,sep = ":")

install.packages("taRifx")
library(taRifx)
maf1_pop_af <- remove.factors(maf1_pop_af)
maf1_pop_af$EAS_AF <- as.numeric(maf1_pop_af$EAS_AF)
maf1_pop_af$max_AF <- pmax(as.numeric(maf1_pop_af$EAS_AF),as.numeric(maf1_pop_af$EUR_AF), as.numeric(maf1_pop_af$AFR_AF), na.rm = T)
summary(maf1_pop_af)

rare_pop_af <- remove.factors(rare_pop_af)
rare_pop_af$EAS_AF <- as.numeric(rare_pop_af$EAS_AF)
rare_pop_af$EUR_AF <- as.numeric(rare_pop_af$EUR_AF)
rare_pop_af$AFR_AF <- as.numeric(rare_pop_af$AFR_AF)
rare_pop_af$max_AF <- pmax(as.numeric(rare_pop_af$EAS_AF), as.numeric(rare_pop_af$EUR_AF), as.numeric(rare_pop_af$AFR_AF), na.rm = T)
summary(rare_pop_af)

rare_pop_af_og[rare_AFR$AFR_AF==2504,]

maf1_SNP <- RESTscreen_attributes$SNP[RESTscreen_attributes$project=="maf1"]
rare_SNP <- RESTscreen_attributes$SNP[RESTscreen_attributes$project=="rare"]

SNP_incl <- c(maf1_SNP,rare_SNP)
pop_AF <- rbind(rare_pop_af, maf1_pop_af)
pop_AF <- pop_AF[which(pop_AF$SNP %in% SNP_incl),]

emvar_comp$maf <- ">1"
emvar_comp$maf[which(emvar_comp$SNP %in% pop_AF$SNP[which(pop_AF$max_AF < 0.01)])] <- "<1"
emvar_comp$maf[which(emvar_comp$SNP %in% pop_AF$SNP[which(pop_AF$max_AF > 0.05)])] <- ">5"
emvar_comp$maf[which(emvar_comp$SNP %in% pop_AF$SNP[which(pop_AF$max_AF > 0.1)])] <- ">10"

emvar_comp$maf <- factor(emvar_comp$maf, levels = c("<1",">1",">5"))

emvar_comp$window <- "outside"
emvar_comp$window[which(emvar_comp$loc_a=="inside" & emvar_comp$loc_r=="inside")] <- "inside"

comp_sil_split <- colsplit(emvar_comp$silencer,":wP",c("sil_allele","wP"))
# emvar_comp$window[which(emvar_comp$loc_a!=emvar_comp$loc_r)] <- ifelse(comp_sil_split$wP[which(emvar_comp$loc_a!=emvar_comp$loc_r)] < 112 & comp_sil_split$wP[which(emvar_comp$loc_a!=emvar_comp$loc_r)] > 90, "inside","outside")
emvar_comp$window <- ifelse(comp_sil_split$wP < 112 & comp_sil_split$wP > 90, "inside","outside")

emvar_comp$rel <- "below"
emvar_comp$rel[which(emvar_comp$score_a >= 20.86 | emvar_comp$score_r >= 20.86)] <- "above"

## Make the plots and calculate statistics by wilcox test comparing window position and relative score

emvar_comp_stat <- list()
for(celltype in unique(emvar_comp$celltype)){
  emvar_comp_stat[[celltype]] <- list()
  message(celltype)
  for(enh in enh_all){
    emvar_comp_stat[[celltype]][[enh]] <- list()
    message(enh)
    for(rel_pos in c("above","below")){
      emvar_comp_stat[[celltype]][[enh]][[rel_pos]] <- list()
      message(rel_pos)
      for(win in c("inside","outside")){
        message(win)
        to_plot <- emvar_comp[which(emvar_comp$celltype==celltype & emvar_comp$enhancer==enh & emvar_comp$maf!=">1" & emvar_comp$window==win & emvar_comp$rel==rel_pos),]
        
        pdf(paste0("plots/minor_allele_frequency_analysis/density/",celltype,"_",enh,"_maf_density_",rel_pos,"_",win,".pdf"), width=40/25.4, height=40/25.4)
        print(ggplot(to_plot, aes(x=LogSkew, group=maf, fill=maf)) + geom_density(adjust=1.5, alpha=0.2, lwd=0.1) + xlim(-5,5) +
                theme_minimal(base_size = 6) + theme(legend.position = "none"))
        dev.off()
        
        to_stat <- remove.factors(to_plot)
        stat_temp <- wilcox_test(to_stat, LogSkew~maf, ref.group = "<1", p.adjust.method = "BH")
        emvar_comp_stat[[celltype]][[enh]][[rel_pos]][[win]] <- stat_temp
        
        # pdf(paste0("plots/minor_allele_frequency_analysis/density/",celltype,"_",enh,"_maf_density_",rel_pos,"_",win,".pdf"), width=40/25.4, height=40/25.4)
        # print(ggplot(to_plot, aes(x=LogSkew, group=maf3b, fill=maf3b)) + geom_density(adjust=1.5, alpha=0.2, lwd=0.1) +
        #         theme_minimal(base_size = 6) + theme(legend.position = "none"))
        # dev.off()
        
      }
    }
  }
}

emvar_comp_stat_df <- data.frame()
for(celltype in names(emvar_comp_stat)){
  message(celltype)
  for(enhancer in names(emvar_comp_stat[[celltype]])){
    message(enhancer)
    for(relativePosition in names(emvar_comp_stat[[celltype]][[enhancer]])){
      message(relativePosition)
      for(window in names(emvar_comp_stat[[celltype]][[enhancer]][[relativePosition]])){
        message(window)
        stat_temp <- data.frame(emvar_comp_stat[[celltype]][[enhancer]][[relativePosition]][[window]])
        stat_temp$celltype <- celltype
        stat_temp$enhancer <- enhancer
        stat_temp$relativePosition <- relativePosition
        stat_temp$window <- window
        emvar_comp_stat_df <- rbind(emvar_comp_stat_df, stat_temp)
      }
    }
  }
}

emvar_comp_stat_df$ks_p <- NA
for(celltype in unique(emvar_comp_stat_df$celltype)){
  message(celltype)
  for(enh in unique(emvar_comp_stat_df$enhancer)){
    message(enh)
    for(rel in unique(emvar_comp_stat_df$relativePosition)){
      message(rel)
      for(win in unique(emvar_comp_stat_df$window)){
        message(win)
        g1 <- emvar_comp$LogSkew[which(emvar_comp$celltype==celltype & emvar_comp$enhancer==enh & emvar_comp$rel==rel & emvar_comp$window==win & emvar_comp$maf3b=="<1")]
        g2 <- emvar_comp$LogSkew[which(emvar_comp$celltype==celltype & emvar_comp$enhancer==enh & emvar_comp$rel==rel & emvar_comp$window==win & emvar_comp$maf3b==">5")]
        
        emvar_comp_stat_df$ks_p[which(emvar_comp_stat_df$celltype==celltype & emvar_comp_stat_df$enhancer==enh & emvar_comp_stat_df$relativePosition==rel & emvar_comp_stat_df$window==win)] <- ks.test(g1,g2)$p.value
      }
    }
  }
}